# konlpy 를 이용한 한국어 텍스트 분석

* 선행작업
    - apt install openjdk python-dev-is-python3 
    - pip install konlpy

In [1]:
from collections import Counter  # 단어 카운트 라이브러리
from wordcloud import WordCloud  # 워드클라우드 라이브러리
from konlpy.tag import Okt       # 트위터 기반 한국어 형태소 분석기 || 이외에도 오픈 라이브러리로 제공되는 Hannanum, kkma, Komoran, Mecab 등이 있음
import nltk                      # nltk 라이브러리

## 워드 클라우드 생성 함수
* 선행작업
    - 한글 출력을 위한 폰트 다운로드(여기에선 D2Coding style)

In [2]:
def wordcloud_from_text(input_file, output_file='../output/wordcloud.png', method=0):
    # get text from file
    try:
        with open(input_file, 'rb') as f:
            text = f.read().decode('utf-8')
    except Exception as e:
        print('wordcloud_from_text() - %s' %(e))
        return
    
    # 예외처리
    if text == None:
        print('wordcloud_from_text() text is None')
        return

    # 명사구 추출
    noun_list = get_noun_list(text, method)
    
    # 예외처리 2
    if len(noun_list) < 10:
        print('wordcloud_from_text() - Too small noun list')
        return
    
    # 워드 클라우드 생성
    wc = WordCloud(font_path = '../../resource/D2Coding-Ver1.3.2-20180524-ligature.ttf',
                       background_color = 'black',
                       width=512, height=512,
                       max_font_size=500,
                       max_words=1000)
    wc.generate_from_frequencies(dict(noun_list))
    wc.to_file(output_file)
    print('Create WordCloud:', output_file)

## 명사구 추출 함수

In [3]:
def get_noun_list(text, method=0):
    # Sentence to token
    if method == 0:
        # 한국어
        noun = tokenizer_konlpy(text)
    else :
        # 영어
        noun = tokenizer_nltk(text)
    
    # Count word
    count = Counter(noun)
    
    # Get most frequent words
    noun_list = count.most_common(3000)
    return noun_list

## Tokenizer

In [4]:
def tokenizer_nltk(text): # 영어
    
    is_noun = lambda pos : (pos[:2] == 'NN' or pos[:2] == 'NNP')
    tokenized = nltk.word_tokenize(text)
    
    return [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]

### 한국어 분석 방법
- 한국어 형태소 분석기에서 주로 사용하게 되는 모듈은 다음과 같다
    - morphs() : 형태소 목록
    - analyze() : 어절단위 형태소 분석(morpheme analysis)
    - pos() : 문장단위 형태소 분석(part-of-speech tagging)
    - nouns() : 명사 단위 어휘 토큰

In [5]:
def tokenizer_konlpy(text): # 한글
    okt = Okt()
    return [word for word in okt.nouns(text) if len(word) > 1 ]

## Main function

In [7]:
if __name__ == '__main__' :
    wordcloud_from_text(input_file = "../../data/Nohs_test.txt", output_file="../../output/Nohs.png", method=0)

Create WordCloud: ../../output/Nohs.png
